In [2]:
import numpy as np

In [2]:
def movie_dict(arr):
    unique, counts = np.unique(arr, return_counts=True)
    k=len(counts)
    L=len(arr)
    movie_id_dict = {}

    for movie_id in arr:
        movie_id_dict[str(movie_id)] = -1
    for i,key in enumerate(movie_id_dict.keys()):
        movie_id_dict[key] = i
    return movie_id_dict
def get_movies_for_user(user,data,movie_mapping):
    data = data[data[:,0]==user]
    movies_user_seen=[]
    for movie in data[:,1]:
        movies_user_seen.append(movie_mapping[str(movie)])
    return movies_user_seen


def one_hot_encoding(data):
    #data = data.astype(int)
    L = data.shape[0]
    N = int(np.max(data[:,0]))
    unique, counts = np.unique(data[:,1], return_counts=True)
    k=len(counts)
    matrix_encoding_user = np.zeros((L,N))
    matrix_encoding_movie = np.zeros((L,k))
    matrix_encoding_other_movies_rated = np.zeros((L,k))
    matrix_encoding_last_movie_rated = np.zeros((L,k))
    movie_mapping = movie_dict(data[:,1])
    matrix_encoding_time = np.array(data[:,3])
    max_time=np.max(matrix_encoding_time)
    min_time=np.min(matrix_encoding_time)
    matrix_encoding_time = ((matrix_encoding_time-min_time)/max_time)

    for i,user in enumerate(data[:,0]):
        matrix_encoding_user[i,int(user)-1] = 1
    for i,movie in enumerate(data[:,1]):
        matrix_encoding_movie[i,movie_mapping[str(movie)]] = 1
    old_user=-1
    for i,user in enumerate(data[:,0]):
        if old_user!=user:
            user_movies = get_movies_for_user(user,data,movie_mapping)
        matrix_encoding_other_movies_rated[i,user_movies] = 1/len(user_movies)
    old_user=-1
    for i,user in enumerate(data[:,0]):
        if old_user==user:
            matrix_encoding_last_movie_rated[i,:] = matrix_encoding_movie[i-1,:]
        old_user=user
    
    
    full_matrix = np.concatenate((matrix_encoding_user,matrix_encoding_movie,matrix_encoding_other_movies_rated,matrix_encoding_time[:, None],matrix_encoding_last_movie_rated),axis=1)

    """ 
    def feed_forward(data,bias,bias_vector,v_matrix):
        N = int(np.max(data[:,0]))
        output = np.zeros(N)
        for i in range(0,N):
            comb_sum = 0
            first_part = bias + np.dot(bias_vector[i],data[i])
            for j in range(1,N):
                factor = np.dot(v_matrix[i],v_matrix[j])
                comb_sum += factor * data[i][j-1] * data[i][j] 
            output[i] =  first_part + comb_sum
        return output
    
    full_matrix = np.concatenate((matrix_encoding_user,matrix_encoding_movie),axis=1)
    print(full_matrix.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_other_movies_rated),axis=1)
    print(full_matrix.shape)
    print(matrix_encoding_time.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_time[:, None]),axis=1)
    print(full_matrix.shape)
    full_matrix = np.concatenate((full_matrix,matrix_encoding_last_movie_rated),axis=1)
    """
    return full_matrix



In [3]:
from scipy import sparse
filename="ml-latest-small/ratings.csv"
data=np.genfromtxt(filename,skip_header=1,delimiter=",")
unique, counts = np.unique(data[:,1], return_counts=True)
full_matrix_test = one_hot_encoding(data)
full_matrix_sparse = sparse.csr_matrix(full_matrix_test,dtype=float)



In [18]:
#Need to fix
genres_filename="ml-latest-small/movies.csv"
f = open(genres_filename, "r")
f.readline()
nrMovies=x = len(f.readlines())
f = open(genres_filename, "r")
f.readline()
genre_dict={"unknown":0,"Action":1,"Adventure":2,"Animation":3,"Children":4,"Comedy":5,
"Crime":6,"Documentary":7,"Drama":8,"Fantasy":9,"Film-Noir":10,"Horror":11,"Musical":12,
"Mystery":13,"Romance":14,"Sci-Fi":15,"Thriller":16,"War":17,"Western":18,"IMAX":19}
movie_genre_dict={}
nrexcluded=0
genre_matrix=np.zeros((nrMovies,len(genre_dict)))
for line in f.readlines():
    movie_ind = line.split(",")[0]
    genres_string = line.split(",")[-1]
    genres_string=genres_string[:-1]
    movie_encoded=np.zeros((1,len(genre_dict)))
    for genre in genres_string.split("|"):
        try:
            movie_encoded[0,genre_dict[genre]] = 1
            movie_genre_dict[movie_ind]=movie_encoded
        except:
            nrexcluded=nrexcluded+1
print("Nr. exluding",nrexcluded)

Nr. exluding 34


In [4]:
full_matrix_test.shape
sparse.save_npz('ml_latest_small_formated.npz', full_matrix_sparse)
#print(len(counts))